<p style="background-color:#808080; text-align: center;">
    <img src="https://www.kitepackaging.co.uk/images/kite-logo-2021.png" alt="Logo" width="100" height="200">
</p>


<h1 style="text-align: center; font-weight: bold;">
    KITE PACKAGING : GENERATING DISTANCES &nbsp; 
</h1>
           

<div style="text-align: center;">
    <img src="https://media.kitepackaging.co.uk/images/content/map/branchmap2022.jpg" alt="Branch Map" width="200" height="100">
</div>


<p style="background-color:#808080; text-align: center;">
    <img src="https://www.kitepackaging.co.uk/images/kite-logo-2021.png" alt="Logo" width="100" height="200">
</p>


### 1. Using Postcodes.io API to Generate Coordinates for Partial Postcodes

>**Note**  :
>
- Most postcodes in the original postcodes.csv file are partial, containing a full outward code (e.g., "BE2") and, in most cases, the first digit of the inward code (e.g., "BE2 2"). However, some entries lacked a space between these sections (e.g., "BE22" instead of "BE2 2").
- To ensure accurate distance calculations, we assumed entries without spaces were missing formatting due to spacing or data entry errors. For example, "BE22" was treated as "BE2 2," assuming the outward code is followed by the first digit of the inward code, as observed in similar entries.
- This manually reviewed and reorganised file is saved as a new file, customer_postcodes.csv, to generate coordinates as shown below.


In [4]:
import pandas as pd
import requests
import time

# Load the CSV file containing the partial postcodes
input_file = 'customer_postcodes.csv'  
output_file = 'postcodes_coordinates.csv'  

# Read the CSV file into a DataFrame
df = pd.read_csv(input_file)

# Create a copy of the DataFrame to store results, including all original columns
results_df = df.copy()

# Add new columns for Latitude and Longitude, initialized with None in the copy
results_df['Latitude'] = None
results_df['Longitude'] = None

# Function to get latitude and longitude using Postcodes.io API
def get_coordinates(postcode):
    # Extract the outward code (first part of the postcode)
    outcode = postcode.split()[0]
    url = f'https://api.postcodes.io/outcodes/{outcode}'
    response = requests.get(url)
    
    # If the API request is successful
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 200:
            latitude = data['result']['latitude']
            longitude = data['result']['longitude']
            return latitude, longitude
        else:
            return None, None
    else:
        return None, None

# Iterate through each row in the DataFrame and get coordinates for each postcode
for index, row in df.iterrows():
    postcode = row['DeliveryPostCode']  # Use the DeliveryPostCode column
    
    # Get latitude and longitude for the postcode
    latitude, longitude = get_coordinates(postcode)
    
    # Assign the results back to the new DataFrame (results_df)
    results_df.at[index, 'Latitude'] = latitude
    results_df.at[index, 'Longitude'] = longitude
    
    # Optional: Print progress every 100 rows
    if index % 100 == 0:
        print(f'Processed {index} postcodes...')
    
    # Be polite and add a delay to avoid overwhelming the API server
    time.sleep(0.1)

# Save the results to a new CSV file
results_df.to_csv(output_file, index=False)

print(f"Coordinates saved to {output_file}")

Processed 0 postcodes...
Processed 100 postcodes...
Processed 200 postcodes...
Processed 300 postcodes...
Processed 400 postcodes...
Processed 500 postcodes...
Processed 600 postcodes...
Processed 700 postcodes...
Processed 800 postcodes...
Processed 900 postcodes...
Processed 1000 postcodes...
Processed 1100 postcodes...
Processed 1200 postcodes...
Processed 1300 postcodes...
Processed 1400 postcodes...
Processed 1500 postcodes...
Processed 1600 postcodes...
Processed 1700 postcodes...
Processed 1800 postcodes...
Processed 1900 postcodes...
Processed 2000 postcodes...
Processed 2100 postcodes...
Processed 2200 postcodes...
Processed 2300 postcodes...
Processed 2400 postcodes...
Processed 2500 postcodes...
Processed 2600 postcodes...
Processed 2700 postcodes...
Processed 2800 postcodes...
Processed 2900 postcodes...
Processed 3000 postcodes...
Processed 3100 postcodes...
Processed 3200 postcodes...
Processed 3300 postcodes...
Processed 3400 postcodes...
Processed 3500 postcodes...
Proc

In [34]:
# Read the CSV file into a DataFrame
postcodes_coordinates = pd.read_csv('postcodes_coordinates.csv')


In [36]:
# View the first few rows of the dataset.
postcodes_coordinates.head()

,WebsiteID,DeliveryPostCode,DeliveryCountry,Latitude,Longitude
0,759150,AB10 1,United Kingdom,57.135252,-2.120322
1,972698,AB10 6,United Kingdom,57.135252,-2.120322
2,878681,AB11 5,United Kingdom,57.139182,-2.092936
3,973415,AB11 5,United Kingdom,57.139182,-2.092936
4,621179,AB11 9,United Kingdom,57.139182,-2.092936


In [35]:
# Display summary of DataFrame.
postcodes_coordinates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23180 entries, 0 to 23179
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   WebsiteID         23180 non-null  int64  
 1   DeliveryPostCode  23180 non-null  object 
 2   DeliveryCountry   23180 non-null  object 
 3   Latitude          23178 non-null  float64
 4   Longitude         23178 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 905.6+ KB


>**Note**  A few coordinates were missing on the output file, which we manually fetched and added to 'postcodes_coordinates.csv'. For all “EI” postcodes, we assigned a general coordinate for Ireland: 53.41291, -8.24389. Some postcode coordinates outside mainland UK were manually fetched from https://gridreferencefinder.com/.

### 2. Using Google Matrix API to Calculate Distances from Coordinates to CV3 4GB*

In [13]:
import pandas as pd
import googlemaps
import numpy as np

# Initialize Google Maps client with API key
API_KEY = 'MyAPIKey'
gmaps = googlemaps.Client(key=API_KEY)

# Load the data with latitude and longitude coordinates
df = pd.read_csv('postcodes_coordinates.csv')  

# Drop rows with missing Latitude or Longitude
df = df.dropna(subset=['Latitude', 'Longitude'])

# Define the distribution center coordinates for CV3 4GB
origin_coords = "52.378855,-1.493756"  # Coordinates for CV3 4GB

# Function to calculate batched distances
def calculate_batched_distances(origins, destinations):
    try:
        # Call the Google Distance Matrix API with batched destinations
        result = gmaps.distance_matrix(
            origins=origins,
            destinations=destinations,
            mode="driving",
            units="imperial"  # for miles; switch to "metric" for kilometers if needed
        )
        
        # Extract distances in meters and convert to miles and km
        distances_km = []
        distances_miles = []
        for element in result['rows'][0]['elements']:
            if 'distance' in element:
                distance_meters = element['distance']['value']
                distances_km.append(distance_meters / 1000)  # Convert meters to km
                distances_miles.append(distance_meters * 0.000621371)  # Convert meters to miles
            else:
                distances_km.append(None)
                distances_miles.append(None)
        
        return distances_km, distances_miles
    except Exception as e:
        print(f"Error with batch request: {e}")
        return [None]*len(destinations), [None]*len(destinations)

# Prepare destination coordinates by combining Latitude and Longitude
df['DestinationCoords'] = df['Latitude'].astype(str) + ',' + df['Longitude'].astype(str)

# Split destination coordinates into batches of 25
batch_size = 25
destination_batches = [df['DestinationCoords'][i:i + batch_size].tolist() for i in range(0, len(df), batch_size)]

# Initialize lists to store all results
all_distances_km = []
all_distances_miles = []

# Loop over each batch and calculate distances
for batch in destination_batches:
    distances_km, distances_miles = calculate_batched_distances(origin_coords, batch)
    all_distances_km.extend(distances_km)
    all_distances_miles.extend(distances_miles)

# Add the calculated distances back to the original DataFrame
df['Distance_km_new'] = all_distances_km
df['Distance_miles_new'] = all_distances_miles

# Save the updated DataFrame with the new distances to a single CSV file
df.to_csv('postcodes_coord_dist.csv', index=False)

print("Batched distances calculated and saved to ''postcodes_coord_dist.'")


Batched distances calculated and saved to ''postcodes_coord_dist.'


In [5]:
import pandas as pd

# Read the CSV file into a DataFrame
postcodes_coord_dist = pd.read_csv('postcodes_coord_dist.csv')

In [7]:
# View the first few rows of the dataset.
postcodes_coord_dist.head()

,WebsiteID,DeliveryPostCode,DeliveryCountry,Latitude,Longitude,DestinationCoords,Distance_km_new,Distance_miles_new
0,573916,CV3 1,United Kingdom,52.391076,-1.478034,"52.39107625,-1.478033608",3.571,2.218916
1,584793,CV3 1,United Kingdom,52.391076,-1.478034,"52.39107625,-1.478033608",3.571,2.218916
2,614840,CV3 1,United Kingdom,52.391076,-1.478034,"52.39107625,-1.478033608",3.571,2.218916
3,615651,CV3 1,United Kingdom,52.391076,-1.478034,"52.39107625,-1.478033608",3.571,2.218916
4,668776,CV3 1,United Kingdom,52.391076,-1.478034,"52.39107625,-1.478033608",3.571,2.218916


In [9]:
# Display summary of DataFrame.
postcodes_coord_dist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23178 entries, 0 to 23177
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   WebsiteID           23178 non-null  int64  
 1   DeliveryPostCode    23178 non-null  object 
 2   DeliveryCountry     23178 non-null  object 
 3   Latitude            23178 non-null  float64
 4   Longitude           23178 non-null  float64
 5   DestinationCoords   23178 non-null  object 
 6   Distance_km_new     23177 non-null  float64
 7   Distance_miles_new  23177 non-null  float64
dtypes: float64(4), int64(1), object(3)
memory usage: 1.4+ MB


In [17]:
# Count the total number of missing values in the DataFrame
null_values_count = postcodes_coord_dist.isnull().sum().sum()

# Get columns with missing values
columns_with_null = postcodes_coord_dist.columns[postcodes_coord_dist.isnull().any()].tolist()

# Get row and column indices of null values
null_locations = postcodes_coord_dist.isnull().stack()
null_locations = null_locations[null_locations].index.tolist()

# Format the output message
output_message = f"The total number of null values in the DataFrame is: {null_values_count}\n"
if columns_with_null:
    output_message += f"Missing values found in the following columns: {', '.join(columns_with_null)}\n"
    output_message += "Null values located at:\n"
    for loc in null_locations:
        delivery_postcode = postcodes_coord_dist.at[loc[0], "DeliveryPostCode"]
        output_message += f"Row: {loc[0]}, Column: {loc[1]}, DeliveryPostCode: {delivery_postcode}\n"
else:
    output_message += "No columns have null values."

print(output_message)


The total number of null values in the DataFrame is: 2
Missing values found in the following columns: Distance_km_new, Distance_miles_new
Null values located at:
Row: 23177, Column: Distance_km_new, DeliveryPostCode: TR22 0
Row: 23177, Column: Distance_miles_new, DeliveryPostCode: TR22 0



## Final Postcode File

>**Note**  : 
The distance for postcode 'TR22 0' wasn’t picked up by the Google Maps API – likely due to its location in the Isles of Scilly. We have manually added this to the 'postcodes_coord_distance.csv'.

In [4]:
import pandas as pd

# Read the CSV file into a DataFrame
postcodes_final = pd.read_csv('postcodes_coord_distance.csv')

In [6]:
# View the first few rows of the dataset.
postcodes_final.head()

,WebsiteID,DeliveryPostCode,DeliveryCountry,Latitude,Longitude,Distance_km,Distance_miles
0,245,L39 2,United Kingdom,53.561637,-2.896693,200.136,124.359
1,597,PR3 1,United Kingdom,53.867059,-2.719333,231.076,143.584
2,739,CV2 2,United Kingdom,52.424559,-1.463448,10.301,6.401
3,993,W1W 8,United Kingdom,51.519216,-0.140903,149.528,92.912
4,994,LE15 8,United Kingdom,52.656127,-0.702324,89.569,55.656


In [20]:
# Display summary of DataFrame.
postcodes_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23178 entries, 0 to 23177
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   WebsiteID         23178 non-null  int64  
 1   DeliveryPostCode  23178 non-null  object 
 2   DeliveryCountry   23178 non-null  object 
 3   Latitude          23178 non-null  float64
 4   Longitude         23178 non-null  float64
 5   Distance_km       23178 non-null  float64
 6   Distance_miles    23178 non-null  float64
dtypes: float64(4), int64(1), object(2)
memory usage: 1.2+ MB


In [22]:
# Save DataFrame as csv for future use.
postcodes_final.to_csv('postcodes_coord_distance.csv')

>**Note**:The postcodes_coord_distance.csv file will be used in 'Notebook 1' for further review and cleaning, where necessary, before it is used for analysis.